In [10]:
import pandas as pd 
data = pd.read_csv('18-23数据.csv',encoding='utf-8')
data_yichang = pd.read_csv('异常件.csv',encoding='utf-8')
data_yichang['计划员']=0

data = data.append(data_yichang)

data['日期'] = pd.to_datetime(data['日期'], format='%Y%m%d').dt.strftime('%Y-%m-%d')

data = data[(data['日期']>='2023-01-01')&(data['日期']<'2023-07-01')]
data["month"] = pd.to_datetime(data['日期']).dt.month.astype(int)

sku_frequency = data.groupby(['零件号','month']).size().reset_index(name='sku_freq')
sku_frequency.head()

/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_6275/963118126.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(data_yichang)


,零件号,month,sku_freq
0,000979150EC,2,1
1,026109150A,1,1
2,02E301472,1,3
3,02E301472,2,4
4,02E301472,3,6


In [20]:
poc_pred = pd.read_excel('poc第一版数据预测结果数据202309071810.xlsx')
poc_pred['日期'] = pd.to_datetime(poc_pred['月份'], format='%Y%m').dt.strftime('%Y-%m-%d')
poc_pred["month"] = pd.to_datetime(poc_pred['日期']).dt.month.astype(int)
poc_pred = poc_pred[['零件号','month','需求频次abc分类','我们预测','实际']]
poc_pred.head()

,零件号,month,需求频次abc分类,我们预测,实际
0,000098600S,1,F,0.62,0
1,000098600S,2,F,0.57,0
2,000098600S,3,F,0.52,0
3,000098600S,4,F,0.00,0
4,000098600S,5,F,0.00,0


In [22]:
pred_real_freq_demods = pd.merge(poc_pred,sku_frequency,on =['零件号','month'],how ='left')
pred_real_freq_demods['实际'].fillna(0,inplace =True)
pred_real_freq_demods['sku_freq'].fillna(0,inplace =True)
pred_real_freq_demods.head()

,零件号,month,需求频次abc分类,我们预测,实际,sku_freq
0,000098600S,1,F,0.62,0,0.0
1,000098600S,2,F,0.57,0,0.0
2,000098600S,3,F,0.52,0,0.0
3,000098600S,4,F,0.00,0,0.0
4,000098600S,5,F,0.00,0,0.0


In [23]:
pred_real_freq_demods = pred_real_freq_demods[pred_real_freq_demods['实际']!=0]

In [24]:
for i in ['我们预测','实际']:
    pred_real_freq_demods[i]= [1 if x ==0 else x for x in pred_real_freq_demods[i]]
pred_real_freq_demods['mape'] = (abs(pred_real_freq_demods['实际'] - pred_real_freq_demods['我们预测']) 
                                 / pred_real_freq_demods['实际']) 
pred_real_freq_demods.head()

,零件号,month,需求频次abc分类,我们预测,实际,sku_freq,mape
10,000098600T,5,F,1.00,1,0.0,0.000
12,000098601A,1,F,0.11,1,0.0,0.890
25,000098603A,2,F,0.31,2,0.0,0.845
27,000098603A,4,F,0.64,1,0.0,0.360
28,000098603A,5,F,0.41,1,0.0,0.590


In [28]:
pred_real_freq_demods[pred_real_freq_demods['mape']>5].shape

(84, 7)

In [27]:
pred_real_freq_demods.shape

(7887, 7)

In [15]:
pred_real_freq_demods=pred_real_freq_demods[pred_real_freq_demods['mape']<10]

In [16]:
pred_real_freq_demods['fwmape_values']=pred_real_freq_demods['mape']*pred_real_freq_demods['sku_freq']

In [17]:
grouped_mape_info = pred_real_freq_demods.groupby(['month','需求频次abc分类'])['mape'].mean().reset_index()

grouped_wmape_info  = pred_real_freq_demods.groupby(['需求频次abc分类','month'])['实际','我们预测'].sum().reset_index()
grouped_wmape_info['wmape'] = (abs(grouped_wmape_info['实际'] - grouped_wmape_info['我们预测']) / grouped_wmape_info['实际'])

grouped_fwmape_info = pred_real_freq_demods.groupby(['需求频次abc分类','month'])['sku_freq','fwmape_values'].sum().reset_index()
grouped_fwmape_info['fwmape'] = grouped_fwmape_info['fwmape_values'] / grouped_fwmape_info['sku_freq']


grouped_mape_tur_info = pd.merge(grouped_mape_info,grouped_wmape_info[['需求频次abc分类','month','wmape']],on = ['需求频次abc分类','month'],how= 'left')
group_mape_thr_info = pd.merge(grouped_mape_tur_info,grouped_fwmape_info[['需求频次abc分类','month','fwmape']],on = ['需求频次abc分类','month'],how= 'left')

group_mape_thr_info.head()


/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_6275/1703262480.py:3: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped_wmape_info  = pred_real_freq_demods.groupby(['需求频次abc分类','month'])['实际','我们预测'].sum().reset_index()
/var/folders/tq/ty6dg6810571sz60cwyxn8ch0000gn/T/ipykernel_6275/1703262480.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  grouped_fwmape_info = pred_real_freq_demods.groupby(['需求频次abc分类','month'])['sku_freq','fwmape_values'].sum().reset_index()


,month,需求频次abc分类,mape,wmape,fwmape
0,1,A,0.431387,0.097436,0.277379
1,1,B,0.673126,0.014225,0.555761
2,1,C,1.042811,0.189029,0.737986
3,1,D,0.759133,0.015823,0.557670
4,1,E,0.721865,0.098298,0.600103


In [18]:
# 计算mape，wmape 占比情况

# 定义区间
import matplotlib.pyplot as plt
import seaborn as sns

bins = [0, 0.10, 0.30, 0.50, 1.00, 100000]

# 使用cut函数将数据分成不同区间
pred_real_freq_demods['ScoreRange'] = pd.cut(pred_real_freq_demods['mape'], bins=bins, labels=['0-10%', '10%-30%', '30%-50%', '50%-100%', '>100%'],right =False)

pred_real_freq_demods['ScoreRange'].replace('NaN','0',inplace =True)
# 计算每个区间中的数据数量

grouped = pred_real_freq_demods.groupby(['month','ScoreRange']).size().reset_index(name='sku_mm_values')
# 计算每个月份内的总样本数
monthly_counts = pred_real_freq_demods.groupby(['month']).size().reset_index(name='mm_values')

# 合并统计结果
result = pd.merge(grouped, monthly_counts, on=['month'],how = 'left')

# 计算占比
result['Proportion'] = result['sku_mm_values'] / result['mm_values']
result = result[['month','ScoreRange','Proportion']]
result.tail(10)


,month,ScoreRange,Proportion
20,5,0-10%,0.179758
21,5,10%-30%,0.219033
22,5,30%-50%,0.200151
23,5,50%-100%,0.299849
24,5,>100%,0.101208
25,6,0-10%,0.149002
26,6,10%-30%,0.221198
27,6,30%-50%,0.158218
28,6,50%-100%,0.337174
29,6,>100%,0.134409


In [19]:
import pandas as pd

excel_writer = pd.ExcelWriter('poc_pred_info_remove_0.xlsx', engine='xlsxwriter')

# 将每个DataFrame写入不同的工作表
group_mape_thr_info.to_excel(excel_writer, sheet_name='group_info', index=False)
result.to_excel(excel_writer, sheet_name='rate_info', index=False)

# 保存并关闭Excel Writer 对象
excel_writer.save()